In [1]:
pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\abano\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import tkinter as tk
from tkinter import messagebox
import mysql.connector

class InventoryManagementApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Inventory Management System")
        self.root.geometry("800x600")

        self.conn = self.connect_to_database()
        if not self.conn:
            return  
        self.cursor = self.conn.cursor()

        self.create_tables()

        self.main_frame = tk.Frame(self.root)
        self.main_frame.pack(fill="both", expand=True)

        self.menu_bar = tk.Menu(self.root)
        self.root.config(menu=self.menu_bar)

        self.product_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.product_menu.add_command(label="Add Product", command=self.add_product_window)
        self.menu_bar.add_cascade(label="Product", menu=self.product_menu)

        self.supplier_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.supplier_menu.add_command(label="Add Supplier", command=self.add_supplier_window)
        self.menu_bar.add_cascade(label="Supplier", menu=self.supplier_menu)

        self.purchase_order_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.purchase_order_menu.add_command(label="Create Purchase Order", command=self.create_purchase_order_window)
        self.menu_bar.add_cascade(label="Purchase Order", menu=self.purchase_order_menu)

        self.sale_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.sale_menu.add_command(label="Create Sale", command=self.create_sale_window)
        self.menu_bar.add_cascade(label="Sale", menu=self.sale_menu)

    def connect_to_database(self):
        """ Connect to MySQL Database """
        try:
            conn = mysql.connector.connect(
                host="localhost", 
                user="root",  
                password="Abanoub@#$123",  
                database="inventorymanagementsystem"  
            )
            if conn.is_connected():
                print("Connected to MySQL Database")
            return conn
        except mysql.connector.Error as err:
            messagebox.showerror("Connection Error", f"Error: {err}")
            return None

    def create_tables(self):
        """ Create tables if they don't exist """
        try:
            self.cursor.execute(''' 
            CREATE TABLE IF NOT EXISTS Supplier (
                SupplierID INT AUTO_INCREMENT PRIMARY KEY,
                Name VARCHAR(100) NOT NULL,
                ContactNumber VARCHAR(50),
                Email VARCHAR(100),
                Address TEXT
            )''')

            self.cursor.execute(''' 
            CREATE TABLE IF NOT EXISTS Product (
                ProductID INT AUTO_INCREMENT PRIMARY KEY,
                Name VARCHAR(100) NOT NULL,
                Description TEXT,
                Category VARCHAR(50),
                StockLevel INT DEFAULT 0,
                ReorderLevel INT DEFAULT 0,
                PricePerUnit DECIMAL(10, 2) NOT NULL
            )''')

            self.cursor.execute(''' 
            CREATE TABLE IF NOT EXISTS PurchaseOrder (
                OrderID INT AUTO_INCREMENT PRIMARY KEY,
                ProductName VARCHAR(100) NOT NULL,
                Quantity INT NOT NULL,
                SupplierName VARCHAR(100) NOT NULL,
                PricePerUnit DECIMAL(10, 2) NOT NULL
            )''')

            self.cursor.execute(''' 
            CREATE TABLE IF NOT EXISTS Sale (
                SaleID INT AUTO_INCREMENT PRIMARY KEY,
                ProductName VARCHAR(100) NOT NULL,
                Quantity INT NOT NULL,
                CustomerName VARCHAR(100) NOT NULL,
                Date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )''')

            self.conn.commit()
        except mysql.connector.Error as err:
            messagebox.showerror("Database Error", f"Error creating tables: {err}")

    def add_product_window(self):
        self.product_window = tk.Toplevel(self.root)
        self.product_window.title("Add Product")
        self.product_window.geometry("400x300")

        self.create_product_widgets()

    def create_product_widgets(self):
        tk.Label(self.product_window, text="Product Name:").grid(row=0, column=0)
        self.product_name_entry = tk.Entry(self.product_window)
        self.product_name_entry.grid(row=0, column=1)

        tk.Label(self.product_window, text="Description:").grid(row=1, column=0)
        self.product_description_entry = tk.Entry(self.product_window)
        self.product_description_entry.grid(row=1, column=1)

        tk.Label(self.product_window, text="Category:").grid(row=2, column=0)
        self.product_category_entry = tk.Entry(self.product_window)
        self.product_category_entry.grid(row=2, column=1)

        tk.Label(self.product_window, text="Stock Level:").grid(row=3, column=0)
        self.product_stock_entry = tk.Entry(self.product_window)
        self.product_stock_entry.grid(row=3, column=1)

        tk.Label(self.product_window, text="Price per Unit:").grid(row=4, column=0)
        self.product_price_entry = tk.Entry(self.product_window)
        self.product_price_entry.grid(row=4, column=1)

        tk.Button(self.product_window, text="Add Product", command=self.add_product).grid(row=5, columnspan=2)

    def add_product(self):
        product_name = self.product_name_entry.get()
        product_description = self.product_description_entry.get()
        product_category = self.product_category_entry.get()
        stock_level = self.product_stock_entry.get()
        price_per_unit = self.product_price_entry.get()

        if not product_name or not price_per_unit:
            messagebox.showerror("Error", "Product name and price per unit are required!")
            return

        if not stock_level.isdigit() or not price_per_unit.replace('.', '', 1).isdigit():
            messagebox.showerror("Error", "Please enter valid numerical values for stock and price.")
            return

        try:
            self.cursor.execute('''
            INSERT INTO Product (Name, Description, Category, StockLevel, PricePerUnit)
            VALUES (%s, %s, %s, %s, %s)''',
                                (product_name, product_description, product_category, stock_level, price_per_unit))
            self.conn.commit()
            messagebox.showinfo("Success", f"Product '{product_name}' added successfully!")
        except mysql.connector.Error as err:
            messagebox.showerror("Error", f"Error adding product: {err}")

    def add_supplier_window(self):
        self.supplier_window = tk.Toplevel(self.root)
        self.supplier_window.title("Add Supplier")
        self.supplier_window.geometry("400x300")

        self.create_supplier_widgets()

    def create_supplier_widgets(self):
        tk.Label(self.supplier_window, text="Supplier Name:").grid(row=0, column=0)
        self.supplier_name_entry = tk.Entry(self.supplier_window)
        self.supplier_name_entry.grid(row=0, column=1)

        tk.Label(self.supplier_window, text="Contact Number:").grid(row=1, column=0)
        self.supplier_contact_entry = tk.Entry(self.supplier_window)
        self.supplier_contact_entry.grid(row=1, column=1)

        tk.Label(self.supplier_window, text="Email:").grid(row=2, column=0)
        self.supplier_email_entry = tk.Entry(self.supplier_window)
        self.supplier_email_entry.grid(row=2, column=1)

        tk.Label(self.supplier_window, text="Address:").grid(row=3, column=0)
        self.supplier_address_entry = tk.Entry(self.supplier_window)
        self.supplier_address_entry.grid(row=3, column=1)

        tk.Button(self.supplier_window, text="Add Supplier", command=self.add_supplier).grid(row=4, columnspan=2)

    def add_supplier(self):
        supplier_name = self.supplier_name_entry.get()
        contact_number = self.supplier_contact_entry.get()
        email = self.supplier_email_entry.get()
        address = self.supplier_address_entry.get()

        if not supplier_name:
            messagebox.showerror("Error", "Supplier name is required!")
            return

        try:
            self.cursor.execute(''' 
            INSERT INTO Supplier (Name, ContactNumber, Email, Address)
            VALUES (%s, %s, %s, %s)''',
                                (supplier_name, contact_number, email, address))
            self.conn.commit()
            messagebox.showinfo("Success", f"Supplier '{supplier_name}' added successfully!")
        except mysql.connector.Error as err:
            messagebox.showerror("Error", f"Error adding supplier: {err}")

    def create_purchase_order_window(self):
        self.purchase_order_window = tk.Toplevel(self.root)
        self.purchase_order_window.title("Create Purchase Order")
        self.purchase_order_window.geometry("400x300")

        self.create_purchase_order_widgets()

    def create_purchase_order_widgets(self):
        tk.Label(self.purchase_order_window, text="Product Name:").grid(row=0, column=0)
        self.purchase_order_product_name_entry = tk.Entry(self.purchase_order_window)
        self.purchase_order_product_name_entry.grid(row=0, column=1)

        tk.Label(self.purchase_order_window, text="Quantity:").grid(row=1, column=0)
        self.purchase_order_quantity_entry = tk.Entry(self.purchase_order_window)
        self.purchase_order_quantity_entry.grid(row=1, column=1)

        tk.Label(self.purchase_order_window, text="Supplier Name:").grid(row=2, column=0)
        self.purchase_order_supplier_name_entry = tk.Entry(self.purchase_order_window)
        self.purchase_order_supplier_name_entry.grid(row=2, column=1)

        tk.Label(self.purchase_order_window, text="Price per Unit:").grid(row=3, column=0)
        self.purchase_order_price_per_unit_entry = tk.Entry(self.purchase_order_window)
        self.purchase_order_price_per_unit_entry.grid(row=3, column=1)

        tk.Button(self.purchase_order_window, text="Create Order", command=self.create_purchase_order).grid(row=4, columnspan=2)

    def create_purchase_order(self):
        product_name = self.purchase_order_product_name_entry.get()
        quantity = self.purchase_order_quantity_entry.get()
        supplier_name = self.purchase_order_supplier_name_entry.get()
        price_per_unit = self.purchase_order_price_per_unit_entry.get()

        if not product_name or not quantity or not supplier_name or not price_per_unit:
            messagebox.showerror("Error", "All fields are required!")
            return

        if not quantity.isdigit() or not price_per_unit.replace('.', '', 1).isdigit():
            messagebox.showerror("Error", "Please enter valid numerical values for quantity and price.")
            return

        try:
            self.cursor.execute(''' 
            INSERT INTO PurchaseOrder (ProductName, Quantity, SupplierName, PricePerUnit)
            VALUES (%s, %s, %s, %s)''',
                                (product_name, quantity, supplier_name, price_per_unit))
            self.conn.commit()
            messagebox.showinfo("Success", f"Purchase order for '{product_name}' created successfully!")
        except mysql.connector.Error as err:
            messagebox.showerror("Error", f"Error creating purchase order: {err}")

    def create_sale_window(self):
        self.sale_window = tk.Toplevel(self.root)
        self.sale_window.title("Create Sale")
        self.sale_window.geometry("400x300")

        self.create_sale_widgets()

    def create_sale_widgets(self):
        tk.Label(self.sale_window, text="Product Name:").grid(row=0, column=0)
        self.sale_product_name_entry = tk.Entry(self.sale_window)
        self.sale_product_name_entry.grid(row=0, column=1)

        tk.Label(self.sale_window, text="Quantity:").grid(row=1, column=0)
        self.sale_quantity_entry = tk.Entry(self.sale_window)
        self.sale_quantity_entry.grid(row=1, column=1)

        tk.Label(self.sale_window, text="Customer Name:").grid(row=2, column=0)
        self.sale_customer_name_entry = tk.Entry(self.sale_window)
        self.sale_customer_name_entry.grid(row=2, column=1)

        tk.Button(self.sale_window, text="Create Sale", command=self.create_sale).grid(row=3, columnspan=2)

    def create_sale(self):
        product_name = self.sale_product_name_entry.get()
        quantity = self.sale_quantity_entry.get()
        customer_name = self.sale_customer_name_entry.get()

        if not product_name or not quantity or not customer_name:
            messagebox.showerror("Error", "All fields are required!")
            return

        if not quantity.isdigit():
            messagebox.showerror("Error", "Please enter a valid numerical value for quantity.")
            return

        self.update_stock_level(product_name, quantity)
        messagebox.showinfo("Success", f"Sale of '{product_name}' to '{customer_name}' created successfully!")

    def update_stock_level(self, product_name, quantity):
        try:
            self.cursor.execute('''
            SELECT StockLevel FROM Product WHERE Name = %s''', (product_name,))
            stock = self.cursor.fetchone()
            if stock:
                new_stock = stock[0] - int(quantity)
                if new_stock < 0:
                    messagebox.showerror("Error", "Insufficient stock!")
                    return

                self.cursor.execute('''
                UPDATE Product SET StockLevel = %s WHERE Name = %s''', (new_stock, product_name))
                self.conn.commit()
            else:
                messagebox.showerror("Error", f"Product '{product_name}' not found!")
        except mysql.connector.Error as err:
            messagebox.showerror("Database Error", f"Error updating stock level: {err}")

root = tk.Tk()

app = InventoryManagementApp(root)

root.mainloop()


Connected to MySQL Database
